In [1]:
import numpy as np
import pandas as pd



from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import xgboost as xgb

In [2]:
df = pd.read_csv("Analytics Case Study 1-Post Asset Verification.csv")
df

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23
0,0,2/13/2020,ACH,3075,530041,AP,DEALER,SC,0.470588,-0.793282,...,71000,M,OTHERS,SAL,O,0.306122,-0.906781,0.158169,-1.321153,0
1,1,6/2/2020,ADM,3075,532123,AP,DEALER,SC,0.352941,-0.952045,...,51000,F,PG,SAL,O,0.551020,0.163204,1.161336,-1.308269,0
2,2,10/2/2020,ADM,3075,533218,AP,ASC,SC,0.352941,0.318061,...,74038,M,SSC,SEP,O,0.530612,NaN,0.910545,0.605033,0
3,3,10/2/2020,ADM,3075,532484,AP,DEALER,SC,0.352941,-1.309262,...,69720,M,OTHERS,AGR,O,0.224490,-0.167982,-0.544048,-1.098901,0
4,4,10/2/2020,ADM,3075,530046,AP,DEALER,SC,0.352941,-0.713900,...,52220,F,HSC,SAL,O,0.224490,NaN,-0.343415,-0.986164,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11697,11697,2/28/2021,ADM,3052,245101,UP,DEALER,MC,0.470588,0.476825,...,98900,M,SSC,SEP,O,0.122449,-0.448216,-0.243098,0.231391,0
11698,11698,2/28/2021,ACH,3052,250002,UP,ASC,MC,0.588235,0.631619,...,105900,M,OTHERS,SEP,O,0.285714,NaN,0.910545,0.279707,0
11699,11699,2/28/2021,ACH,3052,250401,UP,ASC,MC,0.352941,-1.352922,...,52900,M,HSC,SEP,O,0.306122,NaN,0.057852,0.141202,0
11700,11700,9/2/2021,ACH,3074,244601,UP,ASC,MC,0.470588,0.834042,...,88000,M,OTHERS,SEP,O,0.306122,NaN,-0.092623,0.685559,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11702 entries, 0 to 11701
Data columns (total 23 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      11702 non-null  int64  
 1   V2      11702 non-null  object 
 2   V3      11702 non-null  object 
 3   V4      11702 non-null  int64  
 4   V5      11702 non-null  int64  
 5   V6      11702 non-null  object 
 6   V7      11702 non-null  object 
 7   V8      11702 non-null  object 
 8   V9      11702 non-null  float64
 9   V10     11702 non-null  float64
 10  V11     11702 non-null  float64
 11  V12     11702 non-null  float64
 12  V13     11702 non-null  int64  
 13  V14     11702 non-null  int64  
 14  V15     11698 non-null  object 
 15  V16     11698 non-null  object 
 16  V17     11698 non-null  object 
 17  V18     11698 non-null  object 
 18  V19     11698 non-null  float64
 19  V20     4601 non-null   float64
 20  V21     11702 non-null  float64
 21  V22     11702 non-null  float64
 22

In [4]:
feature_cols = [col for col in df.columns if col not in ['V1','V2','V4', 'V5']]
cat_cols = [col for col in feature_cols if col in ['V2', 'V3', 'V6', 'V7', 'V8', 'V15', 'V16', 'V17', 'V18']]
cont_cols = [col for col in feature_cols if col not in cat_cols]
target_cols = ['V23']

In [10]:
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=2021, stratify=df[target_cols], shuffle=True)

In [11]:
cat_imp = SimpleImputer(strategy='most_frequent')
train_df.loc[:, cat_cols] = cat_imp.fit_transform(train_df[cat_cols])
valid_df.loc[:, cat_cols] = cat_imp.transform(valid_df[cat_cols])

cont_imp = SimpleImputer(strategy='mean')
train_df.loc[:, cont_cols] = cont_imp.fit_transform(train_df[cont_cols])
valid_df.loc[:, cont_cols] = cont_imp.transform(valid_df[cont_cols])

C:\Users\mridu\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\mridu\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\mridu\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [12]:
oe = OrdinalEncoder()
train_df.loc[:, cat_cols] = oe.fit_transform(train_df[cat_cols])
valid_df.loc[:, cat_cols] = oe.transform(valid_df[cat_cols])

C:\Users\mridu\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\mridu\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [13]:
train_df

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23
1694,1694,10/2/2020,1.0,3019,518003,0.0,1.0,2.0,0.352941,0.437134,...,50000.0,1.0,3.0,4.0,1.0,0.673469,0.010349,2.415296,0.566380,0.0
10858,10858,11/19/2020,1.0,3000,600116,20.0,1.0,3.0,0.588235,-0.197919,...,74700.0,1.0,4.0,4.0,1.0,0.857143,-0.346313,-0.092623,-0.989385,0.0
11477,11477,2/25/2021,4.0,3000,600053,20.0,1.0,3.0,0.352941,-0.166167,...,67400.0,1.0,1.0,4.0,1.0,0.612245,-1.288919,0.910545,-0.673723,0.0
1595,1595,1/31/2020,0.0,3004,635105,20.0,0.0,3.0,0.450980,1.230950,...,69900.0,1.0,0.0,4.0,1.0,0.081633,-0.499168,-0.092623,0.666232,0.0
10673,10673,11/17/2020,4.0,3089,151001,17.0,1.0,1.0,0.352941,0.278371,...,52900.0,1.0,2.0,4.0,1.0,0.448980,0.570817,-0.594206,-0.689828,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11136,11136,11/18/2020,4.0,3034,208003,23.0,1.0,3.0,0.352941,-1.309262,...,72900.0,0.0,4.0,4.0,1.0,0.265306,-0.025166,-0.443731,-1.340479,0.0
6251,6251,9/13/2020,3.0,3042,505153,0.0,0.0,2.0,0.352941,2.084303,...,52264.0,1.0,3.0,0.0,1.0,0.306122,-0.025166,-0.092623,2.676165,0.0
2842,2842,7/20/2020,3.0,3075,535270,0.0,1.0,1.0,0.352941,1.389714,...,56000.0,1.0,3.0,0.0,1.0,0.204082,-0.025166,0.057852,1.619662,0.0
11029,11029,11/29/2020,0.0,3064,205001,23.0,1.0,3.0,0.352941,-1.352922,...,52600.0,0.0,4.0,5.0,1.0,0.020408,-0.025166,-0.544048,0.212065,0.0


In [14]:
clf = xgb.XGBClassifier(random_state=42)
clf.fit(train_df[feature_cols], train_df[target_cols])

[22:53:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\mridu\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\mridu\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\mridu\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='binary:logistic', random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

In [16]:
accuracy_score( valid_df[target_cols],clf.predict(valid_df[feature_cols]))

1.0

In [17]:
f1_score( valid_df[target_cols],clf.predict(valid_df[feature_cols]))

1.0

In [18]:
roc_auc_score( valid_df[target_cols],clf.predict_proba(valid_df[feature_cols])[:,1])

1.0